In [1]:
import DirectDmTargets as dddm

calculating effective efficiency for Xe detector done
calculating effective efficiency for Ar detector done
calculating effective efficiency for Ge detector done


In [2]:
import wimprates as wr
assert wr.__version__ !='0.2.2'
wr.__version__

'0.3.1'

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import numericalunits as nu

In [5]:
from tqdm import tqdm

In [6]:
from scipy.integrate import quad as scipy_int

In [7]:
import pandas as pd

In [8]:
import scipy

In [9]:
import emcee
emcee.__version__

'3.0.0'

In [10]:
import corner

In [11]:
import time

# Direct detection of Dark matter using different target materials #

Author:

Joran Angevaare <j.angevaare@nikef.nl>

Date:

14 october 2019 

## Goal ## 

- Roughly reproduce <https://journals.aps.org/prd/abstract/10.1103/PhysRevD.83.083505>
- Update the results thereof with more recent knowledge of the DM-distribution

### Approach ###
To achieve these goals, we must first get a decent recoil spectrum, that is flexible enough to have different astrophysical parameters. Further, it must also be flexible enough to be able to allow for different analytic functions to be added to the model. For instance, we must be able to incorporate the $\rm{SHM}^{++}$ as presented here <https://arxiv.org/abs/1810.11468>.

When we have a sufficiently flexible model, we want to add in the detector physics, we should therein incorporate at least the following parameters:
- target
  - material
  - cross-section
- threshold
- background
- cut efficiency  
- volume
- exposure

Finally we should perform the inference

# Emcee #
<https://emcee.readthedocs.io/en/stable/tutorials/quickstart/>

## Distribution of the DM ##
First we need to make a DM-rate spectrum

In [12]:
stats = dddm.MCMCStatModel("Xe")

StatModel::	initialized for Xe detector. See print(stat_model) for default settings


In [13]:
stats.check_spectrum()

,counts,bin_centers,bin_left,bin_right
0,27.391302,5.0,0.0,10.0
1,208.833173,15.0,10.0,20.0
2,86.353906,25.0,20.0,30.0
3,29.497642,35.0,30.0,40.0
4,9.322992,45.0,40.0,50.0
5,2.674535,55.0,50.0,60.0
6,0.672017,65.0,60.0,70.0
7,0.137943,75.0,70.0,80.0
8,0.019942,85.0,80.0,90.0
9,0.001549,95.0,90.0,100.0


In [14]:
stats.nwalkers = 10
stats.nsteps = 10
stats.log

{'sampler': False, 'did_run': False, 'pos': False}

In [15]:
start = time.time()
stats.run_emcee()
end = time.time()

C:\Users\Joran\programming\Anaconda3\envs\DD_DM\lib\site-packages\emcee\moves\red_blue.py:97: RuntimeWarning: invalid value encountered in double_scalars
  lnpdiff = f + nlp - state.log_prob[j]


emcee: Exception while calling your likelihood function:
  params: [  1.70412596 -44.99530547]
  args: [['log_mass', 'log_cross_section']]
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "C:\Users\Joran\programming\Anaconda3\envs\DD_DM\lib\site-packages\emcee\ensemble.py", line 546, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "C:\Users\Joran\Google Drive\PhD-master\ubuntu-storage\windows-anaconda\DD_DM_targets\DirectDmTargets\statistics.py", line 152, in log_probability
    model = self.eval_spectrum(parameter_vals, parameter_names)
  File "C:\Users\Joran\Google Drive\PhD-master\ubuntu-storage\windows-anaconda\DD_DM_targets\DirectDmTargets\statistics.py", line 220, in eval_spectrum
    return spectrum.get_data(poisson=False)
  File "C:\Users\Joran\Google Drive\PhD-master\ubuntu-storage\windows-anaconda\DD_DM_targets\DirectDmTargets\detector.py", line 173, in get_data
    result['counts'] = self.get_events()
  File "C:\Users\Joran\Google Drive\PhD-master\ubuntu-storage\windows-anaconda\DD_DM_targets\DirectDmTargets\detector.py", line 155, in get_events
    return self.compute_detected_spectrum()
  File "C:

KeyboardInterrupt: 

In [ ]:
end-start

In [ ]:
stats.save_results()

In [ ]:
7028/3600

In [ ]:
assert stats.log['did_run']

In [ ]:
stats.show_walkers()

In [ ]:
stats.show_corner()

## Full dimensionality ##

In [18]:
stats_full = dddm.NestleStatModel("Xe")

StatModel::	initialized for Xe detector. See print(stat_model) for default settings


In [19]:
stats_full.get_summary()

run_nestle::	start_fit for 2 parameters
run_nestle::	fit_done in 3210 s (0.9 h)
	 log_mass  1.72 +/-  0.04
	 mass 52.2 +/- 4.3
	 log_cross_section -45.00 +/-  0.02
	 cross_section 1e-45 +/- 5.5e-47


{'nestle_nposterior': 3395,
 'nestle_time': 3210,
 'nestle_logZ': -11.681566463883755,
 'nestle_logZerr': 0.08328941424200292,
 'summary': 'niter: 11319\nncall: 26436\nnsamples: 12343\nlogz: -11.682 +/-  0.083\nh:  7.104',
 'N_posterior_samples ': 3395,
 'log_mass_fit_res': ' 1.72 +/-  0.04',
 'mass_fit_res': '52.2 +/- 4.3',
 'log_cross_section_fit_res': '-45.00 +/-  0.02',
 'cross_section_fit_res': '1e-45 +/- 5.5e-47'}

In [ ]:
stats_full.log

In [ ]:
# for key in stats_full.log.keys():
#     stats_full.log[key] = False


In [ ]:
stats_full.nwalkers = stats.nwalkers
stats_full.nsteps = stats.nsteps * 2
stats_full.log

In [ ]:
stats_full.fit_parameters = stats_full.known_parameters

In [ ]:
stats_full.check_spectrum()

In [ ]:
start = time.time()
stats_full.run_emcee()
end = time.time()

In [ ]:
end-start

In [ ]:
stats_full.save_results()

In [ ]:
9170.64813709259/3600

In [ ]:
assert stats_full.log['did_run']

In [ ]:
stats_full.show_walkers()

In [ ]:
stats_full.show_corner()

In [ ]:
corner.corner(
    stats_full.sampler.get_chain(
        flat=True,
        thin = 50,
        discard=int(stats_full.nsteps * 0.2)
    ),
              labels=stats_full.fit_parameters);